<a href="https://colab.research.google.com/github/Bhuvanesh-Singla/software-defect-detection/blob/main/DNN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, chi2

In [3]:
!wget https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/jm1.arff
!wget https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/cm1.arff

--2023-10-29 12:07:34--  https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/jm1.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 861589 (841K) [text/plain]
Saving to: ‘jm1.arff’

jm1.arff            100%[===================>] 841.40K  --.-KB/s    in 0.04s   

2023-10-29 12:07:34 (22.4 MB/s) - ‘jm1.arff’ saved [861589/861589]

--2023-10-29 12:07:34--  https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/cm1.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56758 (55

In [4]:
from scipy.io import arff
data = arff.loadarff('jm1.arff')
df= pd.DataFrame(data[0])
df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,b'false'
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,b'true'
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51.0,10.0,8.0,1.0,17.0,36.0,112.0,86.0,13.0,b'true'
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129.0,29.0,28.0,2.0,17.0,135.0,329.0,271.0,5.0,b'true'
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28.0,1.0,6.0,0.0,11.0,16.0,76.0,50.0,7.0,b'true'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,1.0,4.0,52.0,241.48,0.14,7.33,32.93,1770.86,...,13.0,0.0,2.0,0.0,10.0,15.0,30.0,22.0,7.0,b'false'
10881,9.0,2.0,1.0,2.0,30.0,129.66,0.12,8.25,15.72,1069.68,...,5.0,0.0,2.0,0.0,12.0,8.0,19.0,11.0,3.0,b'false'
10882,42.0,4.0,1.0,2.0,103.0,519.57,0.04,26.40,19.68,13716.72,...,29.0,1.0,10.0,0.0,18.0,15.0,59.0,44.0,7.0,b'false'
10883,10.0,1.0,1.0,1.0,36.0,147.15,0.12,8.44,17.44,1241.57,...,6.0,0.0,2.0,0.0,9.0,8.0,21.0,15.0,1.0,b'false'


In [5]:
df.defects.replace({b'false': 0 ,b'true': 1}, inplace = True)

In [6]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy = 'mean').set_output(transform='pandas')
si.fit(df)
df = si.transform(df)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
selector = SelectPercentile(chi2, percentile=80)
selector.fit(df, df['defects'])
mask = selector.get_support()
features = df.iloc[:,mask]

In [9]:
scaler = StandardScaler()
X = features.drop('defects', axis = 1)
y = features.defects

In [10]:
X_scaled = scaler.fit_transform(X)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2)

In [12]:
X_train.shape

(8708, 16)

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam

In [14]:
model = Sequential()

model.add(Dense(512, activation = 'relu', input_shape = (16,1)))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))

model.add(Dropout(0.4))

model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))

model.add(Dropout(0.4))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))

model.add(Dropout(0.2))

model.add(Dense(1, activation = 'sigmoid'))

In [15]:
from keras.src.losses import BinaryCrossentropy
model.compile(optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'], loss = BinaryCrossentropy())

In [16]:
history = model.fit(X_train, y_train, epochs = 100, validation_data= (X_val, y_val))

Epoch 1/100
273/273 [==============================] - 14s 13ms/step - loss: 0.4994 - accuracy: 0.8085 - val_loss: 0.4787 - val_accuracy: 0.7988
Epoch 2/100
273/273 [==============================] - 3s 10ms/step - loss: 0.4715 - accuracy: 0.8091 - val_loss: 0.4814 - val_accuracy: 0.8008
Epoch 3/100
273/273 [==============================] - 3s 10ms/step - loss: 0.4714 - accuracy: 0.8096 - val_loss: 0.4776 - val_accuracy: 0.8002
Epoch 4/100
273/273 [==============================] - 3s 12ms/step - loss: 0.4710 - accuracy: 0.8103 - val_loss: 0.4809 - val_accuracy: 0.8018
Epoch 5/100
273/273 [==============================] - 3s 12ms/step - loss: 0.4692 - accuracy: 0.8099 - val_loss: 0.4770 - val_accuracy: 0.8017
Epoch 6/100
273/273 [==============================] - 3s 12ms/step - loss: 0.4693 - accuracy: 0.8105 - val_loss: 0.4760 - val_accuracy: 0.8016
Epoch 7/100
273/273 [==============================] - 3s 10ms/step - loss: 0.4684 - accuracy: 0.8105 - val_loss: 0.4817 - val_accuracy

In [17]:
test_data = arff.loadarff('cm1.arff')
test_df= pd.DataFrame(test_data[0])
test_df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,b'false'
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,b'true'
2,24.0,5.0,1.0,3.0,63.0,309.13,0.11,9.50,32.54,2936.77,...,1.0,0.0,6.0,0.0,15.0,15.0,44.0,19.0,9.0,b'false'
3,20.0,4.0,4.0,2.0,47.0,215.49,0.06,16.00,13.47,3447.89,...,0.0,0.0,3.0,0.0,16.0,8.0,31.0,16.0,7.0,b'false'
4,24.0,6.0,6.0,2.0,72.0,346.13,0.06,17.33,19.97,5999.58,...,0.0,0.0,3.0,0.0,16.0,12.0,46.0,26.0,11.0,b'false'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,47.0,3.0,1.0,3.0,256.0,1563.78,0.04,28.00,55.85,43785.90,...,2.0,13.0,2.0,0.0,23.0,46.0,144.0,112.0,5.0,b'true'
494,24.0,4.0,3.0,3.0,107.0,587.63,0.05,19.13,30.72,11241.58,...,1.0,7.0,4.0,0.0,22.0,23.0,67.0,40.0,7.0,b'true'
495,82.0,11.0,3.0,10.0,475.0,3155.83,0.02,44.71,70.59,141084.24,...,9.0,59.0,35.0,0.0,32.0,68.0,285.0,190.0,21.0,b'true'
496,10.0,2.0,1.0,1.0,32.0,150.41,0.15,6.50,23.14,977.69,...,1.0,12.0,4.0,0.0,13.0,13.0,19.0,13.0,3.0,b'true'


In [18]:
test_df.defects.replace({b'false': 0 ,b'true': 1}, inplace = True)

In [19]:
selector = SelectPercentile(chi2, percentile=80)
selector.fit(test_df, test_df['defects'])
mask = selector.get_support()
test_features = test_df.iloc[:,mask]

In [20]:
test_features

,loc,v(g),iv(g),n,v,d,i,e,t,lOComment,lOBlank,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,2.0,2.0,1.2,1.2,1.2,1.2,1.4,0
1,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,24.0,5.0,3.0,63.0,309.13,9.50,32.54,2936.77,163.15,0.0,6.0,15.0,15.0,44.0,19.0,9.0,0
3,20.0,4.0,2.0,47.0,215.49,16.00,13.47,3447.89,191.55,0.0,3.0,16.0,8.0,31.0,16.0,7.0,0
4,24.0,6.0,2.0,72.0,346.13,17.33,19.97,5999.58,333.31,0.0,3.0,16.0,12.0,46.0,26.0,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,47.0,3.0,3.0,256.0,1563.78,28.00,55.85,43785.90,2432.55,13.0,2.0,23.0,46.0,144.0,112.0,5.0,1
494,24.0,4.0,3.0,107.0,587.63,19.13,30.72,11241.58,624.53,7.0,4.0,22.0,23.0,67.0,40.0,7.0,1
495,82.0,11.0,10.0,475.0,3155.83,44.71,70.59,141084.24,7838.01,59.0,35.0,32.0,68.0,285.0,190.0,21.0,1
496,10.0,2.0,1.0,32.0,150.41,6.50,23.14,977.69,54.32,12.0,4.0,13.0,13.0,19.0,13.0,3.0,1


In [21]:
X_test = test_features.drop('defects', axis = 1)
y_test = test_features.defects

In [22]:
X_test = scaler.fit_transform(X_test)

In [23]:
model.evaluate(X_test, y_test)

16/16 [==============================] - 1s 9ms/step - loss: 0.3422 - accuracy: 0.8991


[0.34217369556427, 0.8990963697433472]